In [1]:
!pip install intel-extension-for-tensorflow

Defaulting to user installation because normal site-packages is not writeable


In [1]:
!pip install modin

Defaulting to user installation because normal site-packages is not writeable


In [7]:
!pip install --upgrade modin

Defaulting to user installation because normal site-packages is not writeable


In [8]:
import os
os.environ['MODIN_ENGINE'] = 'ray'  # e.g., 'ray' or 'dask'

In [9]:
!pip install ray


Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 MB 105.8 kB/s eta 0:00:0000:0100:20
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.9/97.9 kB 153.4 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 kB 142.1 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 531.0/531.0 kB 141.5 kB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.7/240.7 kB 79.7 kB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 101.5 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 168.8 kB/s eta 0:00:0000:0100:01
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [11]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import LSTM, Dense, Embedding, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
import matplotlib.pyplot as plt
%matplotlib inline
import time
from intel_extension_for_tensorflow import core as intel_ext

2024-02-11 20:39:41.176581: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-11 20:39:41.179721: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-11 20:39:41.210358: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-11 20:39:41.210380: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-11 20:39:41.210407: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to regi

In [4]:
!pip install intel_extension_for_tensorflow_lib

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.3/485.3 MB 91.1 kB/s eta 0:00:0000:0102:16


In [12]:
import modin.pandas as pd
import seaborn as sns

In [13]:
import warnings
warnings.filterwarnings('ignore')

**Reading and Displaying the 4000 stories Dataset**

In [15]:
df = pd.read_csv("4000-Stories-with-sentiment-analysis.csv", sep=',', error_bad_lines=False, index_col=False, dtype='unicode')

TypeError: got an unexpected keyword argument 'error_bad_lines'

In [16]:
df = pd.read_csv("4000-Stories-with-sentiment-analysis.csv",dtype='string')

RuntimeError: Failed to start GCS.  Last 0 lines of error files:

.Please check /tmp/ray/session_2024-02-11_20-48-50_999244_299408/logs/gcs_server.out for details

In [17]:
df.head()

Table 1
NaN    url                                                length title                             text_no author                   story                                              valence           arousal           dominance         text_id                                            processed_text                                     word_count sentence_length  string_length    unique_words lexical_diversity concrete           0.000000 1.000000  2.000000   3.000000 4.000000  5.000000  6.000000  7.000000  8.000000  9.000000  10.000000 11.000000  12.000000  13.000000  14.000000  15.000000  16.000000  17.000000  18.000000  19.000000  20.000000  21.000000  22.000000  23.000000  24.000000  25.000000  26.000000  27.000000  28.000000  29.000000  30.000000  31.000000  32.000000  33.000000  34.000000  35.000000  36.000000  37.000000  38.000000  39.000000  40.000000  41.000000  42.000000  43.000000  44.000000  45.000000  46.000000  47.000000  48.000000  49.000000  50.000000  51.000000  52.000000  53.000000  54.000000  55.000000  56.000000 57.000000  58.000000  59.000000  60.000000  61.000000  62.000000  63.000000  64.000000 65.000000  66.000000  67.000000  68.000000  69.000000 70.000000  71.000000  72.000000  73.000000  74.000000  75.000000  76.000000  77.000000 78.000000  79.000000  80.000000  81.000000  82.000000  83.000000  84.000000  85.000000  86.000000  87.000000  88.000000  89.000000  90.000000  91.000000  92.000000  93.000000 94.000000  95.000000  96.000000  97.000000  98.000000  99.000000  100.000000  101.000000  102.000000  103.000000  104.000000 105.000000  106.000000  107.000000 108.000000  109.000000 110.000000  111.000000  112.000000  113.000000  114.000000  115.000000  116.000000  117.000000  118.000000  119.000000  120.000000  121.000000  122.000000 123.000000  124.000000  125.000000  126.000000  127.000000 128.000000  129.000000  130.000000 131.000000  132.000000  133.000000  134.000000  135.000000  136.000000  137.000000  138.000000  139.000000  140.000000  141.000000  142.000000  143.000000  144.000000  145.000000  146.000000  147.000000  148.000000  149.000000  150.000000  151.000000  152.000000  153.000000  154.000000  155.000000  156.000000  157.000000  158.000000  159.000000  160.000000  161.000000  162.000000  163.000000  164.000000 165.000000  166.000000  167.000000  168.000000  169.000000  170.000000  171.000000  172.000000  173.000000  174.000000  175.000000 176.000000  177.000000  178.000000  179.000000  180.000000 181.000000  182.000000  183.000000  184.000000  185.000000  186.000000  187.000000  188.000000  189.000000  190.000000 191.000000  192.000000  193.000000  194.000000  195.000000  196.000000  197.000000  198.000000 199.000000  200.000000  201.000000  202.000000  203.000000  204.000000  205.000000  206.000000 207.000000  208.000000  209.000000 210.000000  211.000000  212.000000  213.000000  214.000000  215.000000  216.000000  217.000000  218.000000  219.000000  220.000000  221.000000  222.000000  223.000000  224.000000  225.000000  226.000000  227.000000  228.000000 229.000000  230.000000 231.000000 232.000000  233.000000  234.000000  235.000000  236.000000  237.000000  238.000000  239.000000  240.000000 241.000000 242.000000  243.000000  244.000000  245.000000  246.000000  247.000000  248.000000 249.000000  250.000000  251.000000  252.000000 253.000000  254.000000 255.000000  256.000000  257.000000  258.000000  259.000000  260.000000  261.000000  262.000000  263.000000 264.000000  265.000000  266.000000  267.000000  268.000000  269.000000  270.000000  271.000000 272.000000  273.000000  274.000000  275.000000  276.000000  277.000000 278.000000  279.000000  280.000000  281.000000  282.000000  283.000000  284.000000  285.000000  286.000000 287.000000  288.000000  289.000000  290.000000  291.000000  292.000000  293.000000  294.000000  295.000000  296.000000  297.000000  298.000000  299.000000
0.0    https://americanliterature.com/author/eleanor-h... 15044  Peace on Earth, Good-Will to Dogs 0 

In [18]:
df.tail()

Table 1
4066.0 https://americanliterature.com/author/emile-zol... 11547 Captain Burle             4072 Emile Zola CHAPTER I - THE SWINDLE\n\nIt was nine o'clock.... 0.558770117703579 0.384706041316356 0.561164724957963 4072_emile-zola-captain-burle             chapter i the swindle it o'clock the little tow... 5813 20.649373881932  5.60781425972451 1982 0.340959917426458 0.540444518083692 -4.605800 -1.063656  -4.559481  0.320143  2.661729  0.415000  -3.031767  1.698909  5.009296 2.644210  3.576351  0.754999  5.072236 -2.063743  0.343474 -4.988175 -2.761195  0.240081  2.101800 -2.640938 -5.385540  -1.589604 -2.288459 -2.279575 -9.007958 -0.464453  0.353685  1.080624  5.335866  3.751757 -3.365101 -6.260622  1.515899  2.696378  1.307978  4.611014 -0.886034 -1.952022  2.534570  0.117861  2.008025 -3.999286  5.313141  2.584879  2.279197 -2.515825  0.115893 -0.032449 -6.966221 -1.355945  4.076934 -5.806772 -3.714055 -1.591138  2.555290 -4.051523 -2.336746  3.382042  7.481247 -3.856951 -0.610849  1.250891 -0.515857 -0.799694 -1.084446 -2.118278 -2.945320 -4.787571 -1.667265  2.143687 -0.698341 -3.343228 -1.073827  1.614527 -0.300682  5.984507 -0.166129 -5.193346 -0.597202  1.335854 -1.563287 -5.312256 -2.789517  1.483093  5.734973 -2.223661 -5.376008 0.501145 -0.897736  5.004054 -3.539093 3.104928  0.485469 -1.500023 -0.491736 -3.856168  0.600511  2.756955 -1.153910  4.340933  0.366703  1.113922  3.085594 -2.181316  0.875612 -1.793701 -6.917021 1.515977 -7.393713 1.578673 -1.261385 -2.842159 2.503996 -1.368696 -0.239765  0.630947 -6.649915 -2.428307 3.352357  0.748016 5.780280  0.064412 -10.309097 3.239531 -5.703709  0.747556  0.668747 -4.726949  2.500982  3.287122  0.177306  1.524264 -1.036157 -2.629298  2.736484 -1.558362  0.436535 0.365462   4.877294 -0.013149 -2.110846 -0.276257 -0.565093  10.892948 -6.274707  1.307877 -6.260142 -2.733341  2.858218 -2.005029  0.004003  4.356685 -1.743710 2.287802 -2.299540 -1.465269 -1.906386 -1.787144 -0.288152 -0.575155  2.784072 -0.547066  2.320736 -0.035888 -4.297449  1.469165 -1.761343  5.366565 -4.526447 -1.683691 -0.097086 -0.273863 -2.285466  4.545623 3.556428   4.288168 -3.047288  0.958224 -3.336785 -1.596576  4.247891  4.574797 -2.980880 -0.193491 -1.209238  4.120942 -3.939639 4.411232  0.600179 -0.575081 -1.654669 -3.296021 -0.237139  4.593861  1.248172  2.854266 -6.755917 -0.437495 -1.908646 6.040639   5.525450 -4.879155  1.491896 -1.694893  2.917309 -1.814811  5.200338 2.006146  5.110315 -5.448020 -1.902726 -2.834453 -0.437661 1.239629 -1.069405  1.319747 -3.506232 -6.749500 -0.317251  0.286477 11.100542  4.747256 -5.054212  1.255060  0.385979 5.596671 1.065283  2.757303 -6.982468 -2.238748  2.198381  2.611346 -3.037262 -2.625625 1.848442  1.540852 -5.123679 1.051506  1.499539 -5.338203 -1.555724 -5.787939 -0.081322 -6.992154 -1.186930  0.377534  0.691236 1.267351 -5.940706  1.248565  3.552746  1.382239 -2.658167 -0.679988  2.079936  2.614137 -2.518561  -3.088010 3.529507  -2.281617  1.326961 3.277412  0.998315  6.546836 -0.157622 -6.207651 2.149145  0.189342  5.732682 -3.825067 -0.764179 -7.246049 -4.714120 -1.269317 3.688665 3.137380  0.446584 -2.400140  0.120393 -4.581315 0.501602  5.359014 -1.399146 -4.542733 -2.335104  0.926768 -0.051279 -1.498706 -1.400732 -5.802378  2.036475 -1.852210 -2.878432  0.820373  3.333893  1.376029  0.792268  1.079375 -5.319479 -1.326987
4067.0 https://americanliterature.com/author/emile-zol... 10599 Death of Olivier Becaille 4073 Emile Zola CHAPTER I - MY PASSING\n\nIt was on a Saturday,... 0.553441984836173 0.389239101001895 0.553421337665855 4073_emile-zola-death-of-olivier-becaille chapter i my passing it saturday morning i die ... 5381 18.2655172413793 5.40806116669813 1789 0.332466084370935 0.556449881843247  1.365131  0.565146  -2.576051  2.460361  5.204924 -1.873745   1.478540 -6.874128  2.332005 0.101609  1.456919 -1.409010 -3.729502  0.273498  8.472197 -5.723289 -1.000929 -4.485529 -8.511356  7.600939 -6.675573  -4.819704 -1.524681 -2.709115

In [19]:
df.shape

(4072, 1)

In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 4072 entries, (nan, 'url', 'length', 'title', 'text_no', 'author', 'story', 'valence', 'arousal', 'dominance', 'text_id', 'processed_text', 'word_count', 'sentence_length', 'string_length', 'unique_words', 'lexical_diversity', 'concrete', 0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 13.0, 14.0, 15.0, 16.0, 17.0, 18.0, 19.0, 20.0, 21.0, 22.0, 23.0, 24.0, 25.0, 26.0, 27.0, 28.0, 29.0, 30.0, 31.0, 32.0, 33.0, 34.0, 35.0, 36.0, 37.0, 38.0, 39.0, 40.0, 41.0, 42.0, 43.0, 44.0, 45.0, 46.0, 47.0, 48.0, 49.0, 50.0, 51.0, 52.0, 53.0, 54.0, 55.0, 56.0, 57.0, 58.0, 59.0, 60.0, 61.0, 62.0, 63.0, 64.0, 65.0, 66.0, 67.0, 68.0, 69.0, 70.0, 71.0, 72.0, 73.0, 74.0, 75.0, 76.0, 77.0, 78.0, 79.0, 80.0, 81.0, 82.0, 83.0, 84.0, 85.0, 86.0, 87.0, 88.0, 89.0, 90.0, 91.0, 92.0, 93.0, 94.0, 95.0, 96.0, 97.0, 98.0, 99.0, 100.0, 101.0, 102.0, 103.0, 104.0, 105.0, 106.0, 107.0, 108.0, 109.0, 110.0, 111.0, 112.0, 113.0, 114.0, 115.0, 116.0, 11

In [21]:
df.nunique()

Table 1    4030
dtype: int64

In [22]:
df.isnull().sum()

Table 1    0
dtype: int64

In [23]:
df.describe()

,Table 1
count,4072.000000
mean,0.250790
std,5.611812
min,-10.400004
25%,-1.770905
50%,0.200078
75%,2.083567
max,299.000000


In [24]:
df.duplicated().sum()

42

In [25]:
df.drop_duplicates()

Table 1
NaN    url                                                length title                             text_no author                   story                                              valence           arousal           dominance         text_id                                            processed_text                                     word_count sentence_length  string_length    unique_words lexical_diversity concrete           0.000000  1.000000   2.000000   3.000000  4.000000  5.000000   6.000000  7.000000  8.000000  9.000000  10.000000  11.000000  12.000000  13.000000  14.000000  15.000000  16.000000  17.000000  18.000000  19.000000  20.000000  21.000000  22.000000  23.000000  24.000000  25.000000  26.000000  27.000000  28.000000  29.000000  30.000000  31.000000  32.000000  33.000000  34.000000  35.000000  36.000000  37.000000  38.000000  39.000000  40.000000  41.000000  42.000000  43.000000  44.000000  45.000000  46.000000  47.000000  48.000000  49.000000  50.000000  51.000000  52.000000  53.000000  54.000000  55.000000  56.000000  57.000000  58.000000  59.000000  60.000000  61.000000  62.000000  63.000000  64.000000  65.000000  66.000000  67.000000  68.000000  69.000000  70.000000  71.000000  72.000000  73.000000  74.000000  75.000000  76.000000  77.000000  78.000000  79.000000  80.000000  81.000000  82.000000  83.000000  84.000000  85.000000  86.000000  87.000000  88.000000  89.000000  90.000000  91.000000  92.000000  93.000000  94.000000  95.000000  96.000000  97.000000  98.000000  99.000000  100.000000  101.000000  102.000000  103.000000  104.000000  105.000000  106.000000  107.000000  108.000000  109.000000  110.000000  111.000000  112.000000  113.000000  114.000000  115.000000  116.000000  117.000000  118.000000  119.000000  120.000000  121.000000  122.000000 123.000000  124.000000  125.000000  126.000000  127.000000  128.000000  129.000000  130.000000  131.000000  132.000000  133.000000  134.000000  135.000000  136.000000  137.000000  138.000000  139.000000  140.000000  141.000000  142.000000  143.000000  144.000000  145.000000  146.000000  147.000000  148.000000  149.000000  150.000000  151.000000  152.000000  153.000000  154.000000  155.000000  156.000000  157.000000  158.000000  159.000000  160.000000  161.000000  162.000000  163.000000  164.000000  165.000000  166.000000  167.000000  168.000000  169.000000  170.000000  171.000000  172.000000  173.000000  174.000000  175.000000  176.000000  177.000000  178.000000  179.000000  180.000000  181.000000  182.000000  183.000000  184.000000  185.000000  186.000000  187.000000  188.000000  189.000000  190.000000  191.000000  192.000000  193.000000  194.000000  195.000000  196.000000  197.000000  198.000000 199.000000  200.000000  201.000000  202.000000  203.000000  204.000000  205.000000  206.000000 207.000000  208.000000  209.000000  210.000000  211.000000  212.000000  213.000000  214.000000  215.000000  216.000000  217.000000  218.000000  219.000000  220.000000  221.000000  222.000000  223.000000  224.000000  225.000000  226.000000  227.000000  228.000000  229.000000  230.000000  231.000000  232.000000  233.000000  234.000000  235.000000  236.000000  237.000000  238.000000  239.000000  240.000000  241.000000  242.000000  243.000000  244.000000  245.000000  246.000000  247.000000  248.000000  249.000000  250.000000  251.000000  252.000000  253.000000  254.000000  255.000000  256.000000  257.000000  258.000000  259.000000  260.000000  261.000000  262.000000  263.000000  264.000000  265.000000  266.000000  267.000000  268.000000  269.000000  270.000000  271.000000  272.000000  273.000000  274.000000  275.000000  276.000000  277.000000  278.000000  279.000000  280.000000  281.000000  282.000000  283.000000  284.000000  285.000000  286.000000  287.000000  288.000000  289.000000  290.000000  291.000000  292.000000  293.000000  294.000000  295.000000  296.000000  297.000000  298.000000  299.000000
0.0    https://americanliterature.com/author/eleanor-h... 15044  Peac

**Dropping Irrelevant Column from the Dataset**

In [26]:
df.drop(['',''], axis=1, inplace=True)
df

KeyError: "['', ''] not found in axis"

**Explorartory Data Analysis**

In [27]:
correlation = df[['','','','']].corr()
sns.heatmap(correlation,annot=True,cmap='Greens')
plt.title('Correlation')

KeyError: "None of [Index(['', '', '', ''], dtype='object')] are in the [columns]"

In [ ]:
def preprocess_text(text):
    # text cleaning and preprocessing steps
    return processed_text

In [ ]:
processed_text_data = [preprocess_text(text) for text in text_data]

In [ ]:
# Tokenization
tokenizer = Tokenizer(num_words=10000)  
tokenizer.fit_on_texts(processed_text_data)

sequences = tokenizer.texts_to_sequences(processed_text_data)
sequences = keras.preprocessing.sequence.pad_sequences(sequences, maxlen=100)  

word_index = tokenizer.word_index
index_to_word = {index: word for word, index in word_index.items()}

In [ ]:
# LSTM model with oneAPI optimizations
def create_model():
    model = Sequential()
    model.add(Embedding(10000, 128, input_length=100))  
    model.add(LSTM(64, return_sequences=True))  
    model.add(LSTM(32))  
    model.add(Dense(10000, activation='softmax'))

    # Optimize for oneAPI and Gemini architecture using intel_ext.convert
    model = intel_ext.convert(model)

    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

model = create_model()

In [ ]:
# Print the optimized model summary
model.summary()

**Split Data and Train Model**

In [ ]:
!pip install sckit-learn-intelex

In [ ]:
from sklearn import patch_sklearn
patch_sklearn()

In [ ]:
x = df[['','']]

In [ ]:
y = df['']

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.3)

In [ ]:
x_train

In [ ]:
x_test

In [ ]:
y_train

In [ ]:
y_test

**Evaluate Performance**

In [ ]:
test_loss, test_acc = model.evaluate(x_test, y_test)
print('Test accuracy:', test_acc)

**Generate Creative Text**

In [ ]:
# Generate text based on a prompt
def generate_text(prompt, max_length=100, temperature=0.7):
    # Seed the model with the prompt
    seed = tokenizer.texts_to_sequences([prompt])[0]

    for _ in range(max_length):
        # Predict the next word
        probs = model.predict(np.array([seed]))[0]
        next_index = np.random.choice(np.arange)